In [ ]:
import pandas as pd
from sqlalchemy import create_engine
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.cluster import MiniBatchKMeans
from sklearn.preprocessing import StandardScaler

db_connection_str = "postgresql://deepakachyutha@localhost:5432/uscrashdata"
engine = create_engine(db_connection_str)

print("Fetching Post covid ERA Data (2021-2023)")

query = """
SELECT 
    "Severity", "Start_Lat", "Start_Lng", 
    "TemperatureF", "Visibilitymi", "Weather_Condition",
    EXTRACT(HOUR FROM "Start_Time"::timestamp) as "Hour",
    EXTRACT(MONTH FROM "Start_Time"::timestamp) as "Month",
    EXTRACT(ISODOW FROM "Start_Time"::timestamp) as "Weekday"
FROM processed_accidents
WHERE EXTRACT(YEAR FROM "Start_Time"::timestamp) >= 2021
ORDER BY RANDOM()
LIMIT 500000;
"""
df = pd.read_sql(query, engine)
print(f"Loaded {len(df)} rows of modern data.")

print("Processing")


imputer = SimpleImputer(strategy='median')
df[['TemperatureF', 'Visibilitymi']] = imputer.fit_transform(df[['TemperatureF', 'Visibilitymi']])


le = LabelEncoder()
df['Weather_Encoded'] = le.fit_transform(df['Weather_Condition'].astype(str))


print("Clustering Modern Locations")
coords = df[['Start_Lat', 'Start_Lng']]
kmeans = MiniBatchKMeans(n_clusters=50, random_state=42, n_init=10)
df['Region_Cluster'] = kmeans.fit_transform(coords).argmax(axis=1)


scaler = StandardScaler()
df[['Temp_Scaled', 'Vis_Scaled']] = scaler.fit_transform(df[['TemperatureF', 'Visibilitymi']])

final_cols = ['Severity', 'Region_Cluster', 'Weather_Encoded', 
              'Temp_Scaled', 'Vis_Scaled', 'Hour', 'Month', 'Weekday']

df[final_cols].to_csv("modern_training_data.csv", index=False)
print("DONE. 'modern_training_data.csv' is ready for your Final Model.")

Fetching Post covid ERA Data (2021-2023)...


OperationalError: (psycopg2.OperationalError) connection to server at "localhost" (::1), port 5432 failed: Connection refused
	Is the server running on that host and accepting TCP/IP connections?
connection to server at "localhost" (127.0.0.1), port 5432 failed: Connection refused
	Is the server running on that host and accepting TCP/IP connections?

(Background on this error at: https://sqlalche.me/e/20/e3q8)